In [ ]:
from pyspark.sql import SparkSession
from config import MILVUS_CONFIG, SPARK_CONFIG, ivy_cache

In [5]:
def create_spark_session():
    """
    Create SparkSession with Milvus connector JAR and optimized settings
    """
    print("Initializing Spark session with Milvus connector...")
    
    builder = SparkSession.builder \
        .appName(SPARK_CONFIG["app_name"]) \
        .config("spark.jars.repositories", SPARK_CONFIG["repositories"]) \
        .config("spark.jars.packages", 
                           f"{SPARK_CONFIG['maven_coords']}") \
                    .config(
                        "spark.jars.excludes",
                        "org.slf4j:slf4j-api,org.slf4j:slf4j-log4j12,org.slf4j:slf4j-simple"
                    ) \
                    .config("spark.jars.ivy", str(ivy_cache)) \
                    .config("spark.driver.userClassPathFirst", "true") \
                    .config("spark.executor.userClassPathFirst", "true")
    
    # Add additional Spark configurations
    for key, value in SPARK_CONFIG["additional_configs"].items():
        builder = builder.config(key, value)
    
    spark = builder.getOrCreate()
    
    # Set log level to reduce noise
    spark.sparkContext.setLogLevel("WARN")
    
    print(f"Spark version: {spark.version}")
    print(f"Spark application ID: {spark.sparkContext.applicationId}")
    
    return spark

In [ ]:
spark = create_spark_session()

Initializing Spark session with Milvus connector...


25/07/11 16:38:36 WARN Utils: Your hostname, zilliz-Standard-PC-i440FX-PIIX-1996 resolves to a loopback address: 127.0.1.1; using 10.100.32.69 instead (on interface ens18)
25/07/11 16:38:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
file:///home/zilliz/.m2/repository/ added as a remote repository with the name: repo-1
https://repo1.maven.org/maven2/ added as a remote repository with the name: repo-2
https://central.sonatype.com/repository/maven-snapshots/ added as a remote repository with the name: repo-3


:: loading settings :: url = jar:file:/home/zilliz/.spark/spark-3.5.3-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zilliz/.ivy2/cache/cache
The jars for the packages stored in: /home/zilliz/.ivy2/cache/jars
com.zilliz#spark-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf0c2aea-d947-4351-b4e2-1418e440a904;1.0
	confs: [default]
	found com.zilliz#spark-connector_2.13;0.1.6-SNAPSHOT in local-m2-cache
	found com.thesamet.scalapb#scalapb-runtime_2.13;0.11.3 in central
	found com.thesamet.scalapb#lenses_2.13;0.11.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.13;2.4.4 in central
	found com.google.protobuf#protobuf-java;3.15.8 in central
	found io.grpc#grpc-netty-shaded;1.37.0 in central
	found io.grpc#grpc-core;1.37.0 in central
	found io.grpc#grpc-api;1.37.0 in central
	found io.grpc#grpc-context;1.37.0 in central
	found com.google.code.findbugs#jsr305;3.0.2 in local-m2-cache
	found com.google.guava#guava;30.0-android in central
	found com.google.guava#failureaccess;1.0.1 in central
	found com.

Spark version: 3.5.3
Spark application ID: local-1752223123028


NameError: name 'config' is not defined

In [7]:
reader = spark.read.format("milvus")
reader = reader.option("milvus.uri", MILVUS_CONFIG["uri"]) \
                   .option("milvus.token", MILVUS_CONFIG["token"]) \
                   .option("milvus.collection.name", MILVUS_CONFIG["collection_name"]) \
                   .option("milvus.database.name", MILVUS_CONFIG.get("database_name", "default"))
s3_options = {
        "s3.fs": MILVUS_CONFIG.get("s3_filesystem_type", "s3a://"),
        "s3.endpoint": MILVUS_CONFIG["s3_endpoint"],
        "s3.user": MILVUS_CONFIG["s3_access_key"],
        "s3.password": MILVUS_CONFIG["s3_secret_key"],
        "s3.bucket": MILVUS_CONFIG["s3_bucket_name"],
        "s3.rootPath": MILVUS_CONFIG["s3_root_path"],
        "s3.useSSL": MILVUS_CONFIG.get("s3_use_ssl", "true"),
        "s3.pathStyleAccess": MILVUS_CONFIG.get("s3_path_style_access", "false")
    }
for key, value in s3_options.items():
    reader = reader.option(key, value)

milvus_df = reader.load()

Attempting gRPC call for method milvus.proto.milvus.MilvusService/Connect, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/DescribeCollection, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/Connect, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/DescribeCollection, attempt 1


In [8]:
milvus_df.cache()

log4j:WARN No appenders could be found for logger (com.amazonaws.AmazonWebServiceClient).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Attempting gRPC call for method milvus.proto.milvus.MilvusService/Connect, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/GetPersistentSegmentInfo, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/Connect, attempt 1
Attempting gRPC call for method milvus.proto.milvus.MilvusService/GetPersistentSegmentInfo, attempt 1


DataFrame[row_id: bigint, timestamp: bigint, id: bigint, title_vector: array<float>, title: string, link: string, reading_time: bigint, publication: string, claps: bigint, responses: bigint, $meta: string]

In [9]:
milvus_df.printSchema()

root
 |-- row_id: long (nullable = false)
 |-- timestamp: long (nullable = false)
 |-- id: long (nullable = false)
 |-- title_vector: array (nullable = false)
 |    |-- element: float (containsNull = true)
 |-- title: string (nullable = false)
 |-- link: string (nullable = false)
 |-- reading_time: long (nullable = false)
 |-- publication: string (nullable = false)
 |-- claps: long (nullable = false)
 |-- responses: long (nullable = false)
 |-- $meta: string (nullable = true)



In [10]:
milvus_df.show(5)

+------------------+------------------+---+--------------------+--------------------+--------------------+------------+-----------+-----+---------+-----+
|            row_id|         timestamp| id|        title_vector|               title|                link|reading_time|publication|claps|responses|$meta|
+------------------+------------------+---+--------------------+--------------------+--------------------+------------+-----------+-----+---------+-----+
|459307002669264412|459307579105083393|  0|[0.041732933, 0.0...|The Reported Mort...|https://medium.co...|          13|The Startup| 1100|       18|   {}|
|459307002669264413|459307579105083393|  1|[0.0039737443, 0....|Dashboards in Pyt...|https://medium.co...|          14|The Startup|  726|        3|   {}|
|459307002669264414|459307579105083393|  2|[0.031961977, 4.7...|How Can We Best S...|https://medium.co...|           6|The Startup|  500|        7|   {}|
|459307002669264415|459307579105083393|  3|[0.032572296, -0....|Maternity le

In [11]:
spark.stop()